# Makine Öğrenmesi-2

**1-) Giriş**

Makine öğrenimi notebook'un ikinci kısmına hoş geldiniz. Github üzerindeki birinci notebook'u çalıştıktan sonra buraya bakmanızı tavsiye ederim. Bu notebook ile modellerimizin kalitesini hızlı bir şekilde artıracağız. Gerçek veriler üzerinde işlem yapacağız ve modellerimizi doğrulamak için gelişmiş teknikleri kullanacağız. Yaygın olarak kullanılan son teknoloji modellere göz atacağız. ( XGBoost vs.)

**2-) Eksik Değerler ( Missing Values )**

* Eksik değerlere sahip olan sutünları silme, 
* Imputation: Eksik değerleri bir sayı ile doldurur. 

**Uygulama**
Örnekte, Melbourne konut veri kümesiyle çalışacağız. Modelimiz, konut fiyatını tahmin etmek için oda sayısı ve arazi büyüklüğü gibi bilgileri kullanacaktır.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# verinin yüklenmesi.
data = pd.read_csv('melb_data.csv')

# hedef
y = data.Price

# sayısal değerleri kullanacağız.
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# datasetimizi train ve valid olarak ayıralım.
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [2]:
# veri temizleme yaklaşımlarının hepsini inceleyelim.
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# farklı yaklaşımları karşılaştırmak adına tanımladığımız fonksiyon.
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [5]:
#YÖNTEM1

# eksik değerlere sahip sütunların isimlerini alalım.
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# eğitim ve doğrulama verilerinde eksik değerler içeren sutunları silelim.
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("1. Yaklaşımdan MAE (Eksik değerlere sahip sutünları sil.):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

1. Yaklaşımdan MAE (Eksik değerlere sahip sutünları sil.):
354257.66157608695


Eksik değerlerin tamamını sildikten sonra elde ettiğimiz değer 354257. Amacımız bu puanı düşürmek. Şimdi ise ikinci yöntemimizi deneyeceğiz. Eksik değerleri silmek yerine eksik değerin bulunduğu sütunun ortalamasını eksik değerin olduğu alana yazacağız. Bu işlem daha iyi sonuç verebilir. Ancak tabiki de veri setimize göre değişkenlik gösterebilir.İmputed değerlerini belirlemek adına daha karmaşık yollar seçilsede ( regression imputation ) gibi sonuçları makine öğrenme modellerine bağladığımızda karmaşık stratejiler genelde ek bir fayda sağlamaz.

In [6]:
#YÖNTEM2

from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# İmputation tarafından kaldırılan sutün adlarını tekrardan yazalım.
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("2. Yaklaşımdan MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

2. Yaklaşımdan MAE (Imputation):
203078.71828804348


Yöntem 2'nin Yöntem 1'den daha düşük MAE değerine ( Mean Absolute Error  ) sahip olduğunu görüyoruz, bu nedenle Yöntem 2 bu veri kümesinde daha iyi performans gösterdi.

In [8]:
# BONUS YÖNTEM - (An Extension to Imputation)
# Bu yöntem imputation ile eksik değerleri ifade ederken, hangi değerlerin ifade edildiğini de takip ediyoruz.

# orjinal verileri değiştirmemek adına kopyalama yaptık.
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# neyin uygulanacağını gösteren yeni sütunlar oluşturduk.
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation ile silinen sutünların isimlerini tekrardan oluşturalım.
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("2. Yaklaşımdan MAE (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

2. Yaklaşımdan MAE (An Extension to Imputation):
202839.1169021739


**3-) Categorical Variables - Kategorik Değişkenler** 

Kategorik bir değişken yalnızca sınırlı sayıda değerler alır. Örneğin ne sıklıkla kahvaltı yaptığınızı soran ve sadece 4 seçenek bulunan anketi düşünelim. Anket seçenekleri 'Asla', 'Nadiren', 'Çoğu Zaman' ve 'Hergün' olurdu. Bu durumda veriler kategoriktir. Çünkü yanıtlar sabit bir kategoriye girer. Ya da insanların hangi model araç kullandığı verisi de kategoriktir. Bu verileri makine öğrenmim algoritmalarına direkt olarak verirsek hata alırız. Bu bölümde kategorik verileri makine öğrenim algoritmalarına hazırlamak için 3 temel adıma göz atacağız.


* *1) Kategorik Verileri Silin.*
* *2) Sıralı Encoding:* Bu işlem her benzersiz değeri bir tamsayıya atar. Makine öğrenim algoritmalarından ağaç tabanlı modeller için ( karar ağaçları , rastgele ormanlar ) sıralı kodlamanın sıralı değişkenlerde iyi çalıştığı gözlemlenmiştir.
* *3) One-Hot Encoding:* Tek satırlı kodlama, orijinal verilerdeki olası her değerin varlığını (veya yokluğunu) gösteren yeni sütunlar oluşturur. Mantığı şudur: Benzersiz kolon değerleri ile matris oluşturulur. Satır ve sutunu aynı olan değerler 1, diğerleri 0 değerini alır.Sıralı kodlamanın aksine, tek satırlı kodlama kategorilerin sırasını varsaymaz. Bu nedenle, kategorik verilerde net bir sıralama yoksa bu yaklaşımın özellikle iyi çalışmasını bekleyebilirsiniz. İçsel sıralaması olmayan kategorik değişkenleri *nominal değişkenler* olarak adlandırırız.Kategorik değişken çok sayıda değer alırsa (yani, genellikle 15'ten fazla farklı değer alan değişkenler için), tek satırlı kodlama genellikle iyi performans göstermez.

**Uygulama**

Önceki eğitimde olduğu gibi, Melbourne Konut veri seti ile çalışacağız.

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split

# datanın yüklenmesi.
data = pd.read_csv('melb_data.csv')

# görev dağılımı.
y = data.Price
X = data.drop(['Price'], axis=1)

# datasetin eğitim/doğrulama olarak ayrılması.
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# kayıp değerlerin silinmesi.
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinality" bir sutündaki benzersiz değerler anlamına gelir.
# benzersiz değerlere sahip sutünların seçimi.
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# sayısal olan sütunların seçimi.
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# yalnızca seçili olan sütunları sakla. çünkü kategorik veriler ile işlem yapacağız.
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [23]:
# head kullanarak eğitim verilerine şöyle bir göz atalım.
X_train.head()

,Type,Method,Unnamed: 0,Rooms
2573,h,SP,3349,4
2091,h,SP,2686,3
4683,u,S,6065,2
8832,h,VB,11346,3
10469,u,S,13474,2


In [24]:
# şimdi ise eğitim verilerindeki tüm kategorik verilerin listesini alalım. Bunu her sütunun veri tipini kontrol ederek yaparız.
# nesne türü bir sütunun metin olduğunu gösterir. bu verisetimizde metin içeren sütünlar bizim için kategorik değişkendir.

# eğitim setimizdeki kategorik değişkenleri liste şeklinde alalım.
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("kategorik değişkenler:")
print(object_cols)

kategorik değişkenler:
['Type', 'Method']


In [25]:
# bir önceki örnekte yaptığımız gibi yöntemlerimizi denemek adına fonksiyonumuzu tanımlıyoruz.
# aynı şekilde bu fonksiyon aracılığı ile MAE hesabını yapacağız ve en düşük hata değerini yakalamaya çalışacağız.

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# karşılaştırma yapabilmemiz adına yöntemlerin skolarlarını elde edeceğimiz fonksiyonumuz.
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)


In [26]:
# YÖNTEM 1: select_type ile tipi object yani string yani kategorik değer içeren bütün sutunları siliyoruz.

drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("1. yöntem için MAE değeri: (kategorik değerlerin silinmesi yöntemi)")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

1. yöntem için MAE değeri: (kategorik değerlerin silinmesi yöntemi)
345278.21768750006


In [27]:
# YÖNTEM 2: Ordinal Encoding - Sıralı Kodlama- Bu işlem için sklearn'in OrdinalEncoder paketini import edeceğiz.

from sklearn.preprocessing import OrdinalEncoder

# orjinal datamız bozulmasın diye kopyalayalım. 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# kategorik veriler içeren her sütuna ordinal encoder uygulayalım.
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("2. yöntem için MAE değeri: (Ordinal Encoding yöntemi)") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

2. yöntem için MAE değeri: (Ordinal Encoding yöntemi)
314139.64080978255


In [28]:
# YÖNTEM 3: One-Hot Encoding - Bu işlem için sklearn'in OneHotEncoder  paketini import edeceğiz.

from sklearn.preprocessing import OneHotEncoder

# kategorik verilerin her sutünuna onehotencoding uyguluyoruz.

# handle_unknown='ignore' parametresini ile doğrulama verileri eğitim verilerinde temsil edilmeyen sınıflar içerdiğinde
# hatalardan kaçınmak için belirledik

# sparse=False parametresi, kodlanmış sütunların numpy dizisi olarak döndürülmesini sağlar.

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# onehot ile kaldırılan sutün isimlerini geri yazalım.
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# kategorik sütunlar kaldırılır.
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# diğer sayısal özelliklere onehot ile kodlanmış sutünları ekleyelim.
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("3. yöntem için MAE değeri: (OneHotEncoding yöntemi)") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

3. yöntem için MAE değeri: (OneHotEncoding yöntemi)
314342.2874402174


Bu dataset için en iyi yöntem , Yöntem 2 oldu. Tabi bu durum datasetler için değişebilir. Dünyamız kategorik veriler ile doludur. Artık kategorik veriler için nasıl çalışacağımızı biliyoruz. İyi bir data science olma yolunda hızla gidiyorsun :)

**4-) Pipelines**

Pipelines veri ön işleme ve modelimizi oluşturduğumuz kodun düzenli kalmasına yardımcı olan en basit tekniklerden bir tanesidir. Veri ön işleme ve modelleme adımlarının bir araya geldiği yerdir. Böylece bu iki adımı da tek bir adımmış gibi kullanabiliriz.

**Uygulama**

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split

# datanın yüklenmesi.
data = pd.read_csv('melb_data.csv')

# görevlendirme.
y = data.Price
X = data.drop(['Price'], axis=1)

# train ve doğrulama olarak verisetinin ayarlanması.
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" bir sutündaki benzersiz değerler anlamına gelir.
# benzersiz değerlere sahip sutünların seçimi.
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# sayısal değerlerin seçimi.
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# # yalnızca seçili olan sütunları sakla. çünkü kategorik veriler ile işlem yapacağız.
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [33]:
X_train.head() # veriler hem kategorik hemde eksik değerler içeriyor. pipelines ile bunların hakkından gelmek kolaydır. :)

,Type,Method,Regionname,Unnamed: 0,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
2573,h,SP,Northern Metropolitan,3349,4,7.8,3058.0,4.0,2.0,1.0,381.0,NaN,1938.0,-37.7337,144.9548,11204.0
2091,h,SP,Southern Metropolitan,2686,3,7.8,3124.0,3.0,1.0,1.0,544.0,160.0,1930.0,-37.8436,145.0581,8920.0
4683,u,S,Southern Metropolitan,6065,2,5.6,3101.0,2.0,1.0,1.0,121.0,NaN,NaN,-37.8126,145.0534,10331.0
8832,h,VB,Southern Metropolitan,11346,3,7.5,3123.0,3.0,2.0,2.0,200.0,NaN,NaN,-37.8396,145.0514,6482.0
10469,u,S,Southern Metropolitan,13474,2,4.5,3181.0,2.0,1.0,1.0,2842.0,84.0,1920.0,-37.8513,144.9943,7717.0


Pipelines'ı 3 ana adımda inşa ediyoruz:

* *ADIM:1 ÖN İŞLEME ADIMLARINI TANIMLAYIN:*
* Farklı ön işleme adımlarını bir araya getirmek için Column Transforme sınıfını kullanırız.
* Aşağıda yazdığımız kod bloğu sayısal verilerdeki eksik değerleri ifade eder ve kategorik verilere onehot enconding i uygular.

In [34]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# sayısal veriler için ön işleme.
numerical_transformer = SimpleImputer(strategy='constant')

# kategorik veriler için ön işleme.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# sayısal ve kategorik veriler için paket ön işleme.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

* *ADIM:2 MODELİ TANIMLAYIN:*
* RandomForestRegressor sınıfıyla rastgele bir orman modeli ( random forest ) tanımlarız.

In [35]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

* *PIPELINE'I OLUŞTURMA VE DEĞERLENDİRME:*
* Son olarak ön işleme ve modelleme adımlarını birleştiren pipeline i tanımlamak adına Pipeline class'ını kullanacağız. Dikkat edilmesi gereken bir kaç önemli nokta var.
    *Pipeline ile eğitim verilerini önceden işleriz.Modeli tek bir kod satırına sığdırırız. ( Bu karşılık eğer pipeline kullanmasaydık imputation, onehotencoding ve model eğitimi ayrı adımlar ile yapılması gerekecekti. Hem kategorik hemde sayısal veriler ile uğraşmak yazdığımız kodu bir hayli karışık hale getirecekti. ) 
    *Pipeline ile X_valid içindeki işlenmemiş özellikleri predict() komutuna veririz. Böylece pipeline tahminler oluşturulmadan önce özellikleri otomatik olarak önceden işler.

In [36]:
from sklearn.metrics import mean_absolute_error

# paket ön işleme ve modelleme kodumuzu buraya aldık.
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# eğtim verilerini ön işlemeden geçirip akabinde eğitiyoruz.
my_pipeline.fit(X_train, y_train)

# doğrulama verilerini ön işlemeden geçirip akabinde tahminler üretiyoruz.
preds = my_pipeline.predict(X_valid)

# modelimizi değerliyoruz.
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 177453.58753804347


**5-) Cross-Validation**

Bu bölümde modelimizin performansını daha iyi ölçmek adına çapraz doğrulamayı (cross validation) öğreneceğiz.

Makine öğrenimi yinelemeli bir süreçtir. Doğrulama kümemiz ne kadar büyükse modelin kalitesini ölçtüğümüzde o kadar az rastgelelik vardır ve bir o kadar da güvenilir doğrulamalar alınacaktır.

Çağraz doğrulamada birden fazla model kalitesi ölçümü elde etmek için modelleme sürecimizi datasetimizin farklı alt kümeleri üzerinde çalıştırırız.

Örneğin datasetimizi 5 adet eşit parçaya böldüğümüzü varsayalım.( 5 folds ). Ardından her kat için bir deney yaparız. *Deney 1* de ilk fold'u doğrulama kalan 4 parçayı ise eğitim olarak kullanırız. Sonra *Deney 2* de 2. foldu doğrulama sonra kalan diğer 4 fold'u ise eğitim şeklinde bütün fold'lara test/validation şansı veririz. Bu şekilde verilerin tamamını hem eğitim de hem de validation da kullanmış oluruz.

* Peki cross validation'u ne zaman kullanmalıyız ?

Çapraz doğrulama model kalitesi hakkında daha doğru bir ölçü verir. Bununla birlikte derleme süresi artar. Ekstra hesaplama gücünün önemli olmadığı küçük veri kümelerinde çapraz doğrulama uygulanmaldır. Daha büyük veri kümeleri için tek bir doğrulama kümesi yeterlidir. Alternatif olarak, çapraz doğrulamayı çalıştırabilir ve her denemenin puanlarının yakın görünüp görünmediğini kontrol edebilirsiniz. Her deneme aynı sonuçları verirse, tek bir doğrulama kümesi muhtemelen yeterlidir.

**Uygulama**

In [4]:
import pandas as pd

# datasetin yüklenmesi.
data = pd.read_csv('melb_data.csv')

# alt küme seçimi.
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# görev atama 
y = data.Price

In [5]:
# veri ön işleme ve modelimizi oluşturmak adına pipeline tanımını yapıyoruz.

from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])

In [6]:
# çapraz doğrulama skorlarınu sklearn'ın cross_val_score() fonksiyonu ile elde ederiz. 
# fold sayısını ise cv parametresi ile belirliyoruz.

from sklearn.model_selection import cross_val_score

# sklearn negatif MAE yi hesapladığından dolayı sonucu -1 ile çarptık.
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE skoru:\n", scores)

MAE skoru:
 [330153.84614038 315703.21408613 299410.61884391 243541.74763508
 251301.76683685]


Alternatif modelleri karşılaştırmak için genellikle tek bir model kalitesi ölçüsü istiyoruz. Bu yüzden deneylerin ortalamasını alıyoruz.

In [8]:
print("Ortalama MAE skoru:")
print(scores.mean())

Ortalama MAE skoru:
288022.23870846874


**6-) XGBoost**

Bu bölümde gradient boosting ile modellerimizi nasıl inşa edeceğimizi ve nasıl optimize edeceğimizi öğreneceğiz. Bu yöntem çeşitli veri kümelerinde son teknolojiyi içeren sonuçlar elde eder.

Bu dersten önce genelde makine öğrenim algoritması olarak random forest algoritmasını kullandık. Random forest yöntemlerini **'ensemble method'** olarak adlandırıyoruz. Ensemble method çeşitli modellerin tahminlerini birleştirir.Gradient boosting de  ensemble method grubuna girer.

**Gradient Boosting:** modelleri yinelemeli olarak döngülerden geçiren yöntemdir. Yöntem basit bir tahmin yapan model ile başlar. Veri kümesindeki her gözlem için tahminleme yapmak için geçerli topluluğu kullanırız. Bir tahmin yapmak için topluluktaki tüm modellerin tahminlerini ekleriz. Bu tahminler bir kayıp fonksiyonu hesaplar.( örneğin MAE) Ardından topluluğa eklenecek yeni model için kayıp işlevini kullanırız. Özellikle model parametrelerini belirliyoruz. Böylece yeni modeli topluluğa eklemek genel kaybı azaltacaktır. Bu işlem en az loss değeri elde edilene kadar tekrar edilir.

**Uygulama**

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

# verisetinin yüklenmesi.
data = pd.read_csv('melb_data.csv')

# tahmin için kullanılacak alt kümelerin seçimi.
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# görevlendirme.
y = data.Price

# eğitim ve test olarak verilerin ayrılması.
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

Bu uygulamada XGBoost ile çalışacağız. XGBoost performans ve hıza odaklanan birkaç ek özelliğe sahip gradient boosting'in extreme edilmiş halidir.( sklearn gradient boosting'in özelliğine sahip farklı sürümlerine sahiptir ancak XGBoost'un bazı teknik avantajları vardır. Aşağıdaki kod bloğunda XGBoost için sklearn API'sini içeri aktarıyoruz(xgboost.XGBRegressor). Bu, scikit-learn'de yaptığımız gibi bir model oluşturmamızı ve tahminleme yapmamıza yardımcı olacaktır.(XGBRegressor sınıfının birçok ayarlanabilir parametresi vardır)

In [12]:
from xgboost import XGBRegressor

# modelin oluşturulması:
my_model = XGBRegressor()

# eğitim:
my_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [13]:
# tahminleme:

from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 247486.19087709286


**Parameter Tuning - Parametre Ayarı**

XGBoost'un doğruluğu ve hızını önemli ölçüde etkileyecek çok önemli parametreler mevcuttur. Bu parametreleri anlamak çok çok önemlidir.

* 1-) **n_estimators** : yukarıda açıklanan modelleme döngüsünden kaç kaç yapılacağıdır. topluluğa dahil ettiğimiz model sayısına eşittir. az ya da fazla verilmesi underfitting ve overfittinge neden olurken tipik olarak 10-1000 arasında değişir. ancak bu değer learning rate parametresine birebir bağlıdır.

In [14]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

* 2-) **early_stopping_rounds** : n_estimators için ideal değeri otomatik olarak bulmanın bir yolunu sunar. erken durdurma n_estimators değerine gelmeden validation skor iyileştirmeyi durdurduğunda modelin yinelemeyi durdurmasına neden olur. durdurma gerçekleşmeden önce kaç kez bozulmaya izin verileceği ayarlanır. early_stopping_rounds=5 ideal bir parametredir. bu durumda kötüleşen validation skor üst üste 5 kez meydana geldiğinde yineleme durur. early_stoppıng_rounds kullanırken, doğrulama puanlarını hesaplamak için bazı verileri de ayırmanız gerekir. bu, eval_set parametresini ayarlayarak yapılır.

In [15]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)],
             verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

Daha sonra tüm verilerinizle bir modele eğitmek isterseniz, n_estimators öğesini erken durdurma ile çalıştırıldığında en uygun bulduğunuz değere ayarlayın.

* 3-) **learning_rate**: Genel olarak, küçük bir öğrenme oranı ve çok sayıda tahminci daha doğru XGBoost modelleri verecektir, ancak döngü boyunca daha fazla yineleme yaptığı için modelin eğitilmesi daha uzun sürecektir. Varsayılan olarak, XGBoost learning_rate=0.1 değerini ayarlar.

In [16]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

* 4-) **n_jobs**: Çalışma zamanının dikkate alındığı daha büyük veri kümelerinde, modellerinizi daha hızlı oluşturmak için paralellik kullanabilirsiniz. n_jobs parametresini makinenizdeki çekirdek sayısına eşit olarak ayarlamak yaygın kullanımıdır. Daha küçük veri kümelerinde genellikle bu parametre kullanılmaz. Ortaya çıkan model daha iyi olmayacak, ancak, fit() komutu sırasında uzun süre bekleyeceğiniz büyük veri kümelerinde kullanışlıdır.

In [17]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

XGBoost, standart tablo verileriyle ( pandas dataframe'inde sakladığınız veri türü) çalışmak için önde gelen bir yazılım kütüphanesidir. Dikkatli parametre ayarlaması ile son derece hassas modelleri eğitebilirsiniz.

**7-) Data Leakage**

Bu bölümde modelimizi bozan küçük verileri bulup düzeltmeyi öğreneceğiz. Eğitim verilerimiz hedef hakkında bilgi içerdiğinde veri sızıntısı ( Data leakage (veya leakage)) meydana gelir. Bu durum eğitim hatta doğrulama setinde yüksek performansa yol açarken model gerçek hayatta kötü performans gösterecektir. Başka bir deyişle modelle karar vermeye başlayana kadar modelin doğru görünmesine neden olurken aslında model çok kötüdür. İki ana Data leakage türü vardır. *target leakage* ve *train-test contamination.*

* Target leakage: tahminde bulunduğumuz esnada tahminlerimiz kullanılamayan verileri içerdiğinde oluşur. yani eğitim verilerinde biri değiştiğinde sonuç önemli ölçüde yanlış olarak değişiyorsa meydana gelir.Bu tür veri sızıntısını önlemek için, hedef değer gerçekleştirildikten sonra güncellenen (veya oluşturulan) herhangi bir değişken hariç tutulmalıdır.

* Train-Test Contamination: Eğitim verilerini doğrulama verilerinden ayırmaya dikkat etmediğinizde farklı bir sızıntı türü oluşur. Doğrulama verileri ön işleme davranışını etkiliyorsa, bu işlemi ince yollarla bozabilirsiniz. Buna train-test contamination denir.Örneğin, train_test_split() öğesini çağırmadan önce ön işlemeyi (eksik değerler için bir imputer fit etmek gibi) çalıştırdığınızı düşünün. Sonuç? Modeliniz iyi doğrulama puanları alabilir, bu da size büyük bir güven verebilir, ancak karar vermek için dağıttığınızda kötü performans gösterebilir.Doğrulamanız basit bir train/validation bölünmesine dayanıyorsa, doğrulama verilerini ön işleme adımlarının birleştirilmesi de dahil olmak üzere her türlü birleştirmeden hariç tutun. Scikit-learn pipeline'si kullanırsanız bu daha kolaydır. Çapraz doğrulama kullanırken, ön işleminizi pipeline içinde yapmamız daha da kritik!

**Uygulama**

Kredi kartı uygulamaları hakkında bir veri kümesi kullanacağız. Sonuç olarak, her bir kredi kartı başvurusu hakkındaki bilgiler bir x dataframe'inde saklanır. Bunu, bir seride hangi başvuruların kabul edildiğini tahmin etmek için kullanacağız.

In [21]:
import pandas as pd

# datanın yüklenmesi
data = pd.read_csv('AER_credit_card_data.csv', 
                   true_values = ['yes'], false_values = ['no'])

# görev
y = data.card

# tahmin
X = data.drop(['card'], axis=1)

print("verisetimizdeki satır sayısı:", X.shape[0])
X.head()

verisetimizdeki satır sayısı: 1319


,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5


Bu küçük bir veri kümesi olduğundan, model kalitesinin doğru ölçümlerini sağlamak için çapraz doğrulama kullanacağız.

In [22]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# ön işleme olmadığından bir pipeline a ihtiyacımız yok. ancak yine de yazalım elimiz alışşın :)
my_pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))
cv_scores = cross_val_score(my_pipeline, X, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-validation accuracy: %f" % cv_scores.mean())

Cross-validation accuracy: 0.980292


Verisetimizi incelemek çok önemlidir. Bazı veriler tehlikelidir. Veri sızıntısına neden olabilir.

In [23]:
expenditures_cardholders = X.expenditure[y]
expenditures_noncardholders = X.expenditure[~y]

print('Kart alamayan ve harcaması olmayanların oranı: %.2f' \
      %((expenditures_noncardholders == 0).mean()))
print('Kart alan ve harcaması olmayanların oranı: %.2f' \
      %(( expenditures_cardholders == 0).mean()))

Kart alamayan ve harcaması olmayanların oranı: 1.00
Kart alan ve harcaması olmayanların oranı: 0.02


Yukarıda gösterildiği gibi, kart almayan herkesin harcaması yokken, kart alanların sadece % 2'sinin harcaması yoktu.Ancak harcamaların muhtemelen başvurdukları karttaki harcamalar anlamına geldiği bir hedef sızıntısı durumu gibi görünüyor.


In [24]:
# Veri kümesinden veri sızdıran tahmin etmek için eğiteceğimiz başlıkları silelim.
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis=1)

#bu şekilde modelimizi değerleyelim.
cv_scores = cross_val_score(my_pipeline, X2, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-val accuracy: %f" % cv_scores.mean())

Cross-val accuracy: 0.831683


Evet çapraz doğrulamada değerimiz düştü. Bu istemediğimizi bir durumdu. Ancak veri sızdıran değerleri çıkartmadan önce aldığımız 0.980292 sonuç iyi gibi görünebilir. Ancak veri sızdırma durumu olduğundan gerçek problemlerde iyi çalışmayacaktır.